In [2]:
# This example is based on the tutorial found at https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html

import numpy as np
from qiskit import(QuantumCircuit, execute, Aer, IBMQ, assemble, transpile)
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.ibmq.job import job_monitor
from numpy.random import randint
print("Imports Successful")
# Uncomment when we need to use the actual quantum backends
# provider = IBMQ.load_account()
# provider.backends()

Imports Successful


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') fr

In [15]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def measure_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [19]:
# np.random.seed(seed=0) # This makes the results reproducible, otherwise remove the seed for more random behavior
desired_key_length = 128 # As appropriate for data size and algorithm used
sample_size = 15    # This value can be increased or decreased to affect the chance of finding an eavesdropper 
# With roughly 50% efficiency of transmission, this should produce the desired key length and number of check bits with some buffer to be safe
n = (desired_key_length * 2) + (sample_size * 4) 

## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)

## Step 2
# Create an array to tell us which qubits
# are encoded in which bases
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)

## Step 3
# Decide which basis to measure in:
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)

## Step 4
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

## Step 5
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

print(bob_key)
print(alice_key)
print("Generated key length = %i" % len(alice_key))

  bob_sample = [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
alice_sample = [0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 

In [ ]:
From "Quantum Symmetric Cryptosystem Based on Algebraic Codes", Ali Amerimehr and Massoud Hadian Dehkordi, 2018

Encryption: 
Step 1: Alice computes x := mGk obtaining the messagebits,
and prepares a train of qubits according to x called Qx.
She uses random bases for preparing the qubits.

Step 2: Alice computes h := hk(m), and prepares a
train of qubits according to h called Qh. The base string
{B1,B2, . . .,Br}, used by Alice to prepare Qh, is considered
according to g(x). The rule is that Bj = BZ if the j-th bit of
g(x) is zero, and Bj = BX otherwise.

Step 3: Alice mixes Qx and Qh together according to f(k)
obtaining Qy, and then sends Qy to Bob.

In [18]:
from reedsolo import RSCodec, ReedSolomonError
import hashlib, hmac
import secrets

hash_key = secrets.token_bytes(16)
hash_function = hmac.new(hash_key, digestmod='sha256')
rsc = RSCodec(10)

message = b'hello world'

ecc_encoded = rsc.encode(message)
preshared_key = secrets.token_bytes(len(eccEncoded))

alice_message = bytes(a ^ b for (a, b) in zip(eccEncoded, presharedKey))
hash_function.update(alice_message)
alice_hash = hash_function.digest()

print(alice_message)
print(alice_hash)



b'\x1a\x12\x80\x0edP\xa8(\xa5\xff\x8e\xfc\xd6|6X\xffTE\x04\xb4'
b'\x02\xdd\xba\xd9m\xa2d\x12Q\xc1\x92y\x10\x05\x1d\xd6\xbdA\x18\x03\xdb\x97b\xe8\xe2\x14 \x81\nv\x03k'


In [ ]:
Decryption:

Step 1: Bob receives Qy, and extracts Qx and Qh according
to f(k). Then he measures all qubits of Qx in BY basis
obtaining x, and stores Qh.

Step 2: Bob computes m := x  k, and runs a decoding
algorithm to obtain the message m. (Note that m = mGe,
where e is an error vector arised from Bob’s measurement).
If the error rate is lower than the threshold, he obtains m.
Otherwise, he rejects the communication.

Step 3: Bob first measures the qubits of Qh in the the correct
bases according to g(x) obtaining h. Then he computes h.
If h = h, Bob concludes the message m is valid, and
Alice is legal decisively. Otherwise, Bob does not validate m
(if the quantum channel is noisy d(h, h)  r implies Alice’s
legitimacy, where  is the QBER1 of the channel)

In [30]:
# Working with Binary here:
one_byte = int('11110000', 2)
print(one_byte)

print(bin(22)) # print the binary string for a value

single_byte = one_byte.to_bytes(1, byteorder='big', signed=False) 
print(single_byte)

for i in alice_message:
    print(bin(i)[2:].zfill(8))

240
0b10110
b'\xf0'
00011010
00010010
10000000
00001110
01100100
01010000
10101000
00101000
10100101
11111111
10001110
11111100
11010110
01111100
00110110
01011000
11111111
01010100
01000101
00000100
10110100


In [40]:
from math import sqrt, atan, pi

bob_alpha = sqrt(2 + sqrt(2))/2
bob_beta = sqrt(2 - sqrt(2))/2

# For Bob's basis, w0 = alpha 0 + beta 1, w1 = alpha 0 - beta 1
bam = bob_alpha ** 2
bbm = bob_beta ** 2
bob_w0 = atan(bbm/bam)
bob_w1 = atan(bam/bbm)
print(bob_w1)
bob_22p5_degrees = 0.3926991

# Bob needs a measurement function to measure in this basis
def bob_measure_message(message):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        message[q].rz(pi/8)
        message[q].measure(0,0)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

1.4008778720678356
